# Wymagane biblioteki

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import pandas as pd
import scipy.sparse as sparse
import numpy as np

from tqdm.notebook import tqdm_notebook as tqdm

from gensim.models import Word2Vec


# Przygotowanie danych

## Wczytywanie danych i ich wstepna obrobka

## Obróbka danych pod model ALS

Ze zbioru zostały usuniete gry, których suma godzin była mniejsza lub równa 10. Spowodowane było to błędami programistycznymi.

## Osobna miara
Jako, że sama liczba godzin nie jest w pełni miarodajna co do oceny gry, został zastosowany podział dystrybucji godzin każdej z gier na kwintyle (5 częsci).

In [23]:
steam_raw = pd.read_csv("steam-200k.csv", header=None, names=["User_ID", "Game", "Interaction", "Hours", "Ignore"])
steam_raw

,User_ID,Game,Interaction,Hours,Ignore
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
1,151603712,The Elder Scrolls V Skyrim,play,273.0,0
2,151603712,Fallout 4,purchase,1.0,0
3,151603712,Fallout 4,play,87.0,0
4,151603712,Spore,purchase,1.0,0
...,...,...,...,...,...
199995,128470551,Titan Souls,play,1.5,0
199996,128470551,Grand Theft Auto Vice City,purchase,1.0,0
199997,128470551,Grand Theft Auto Vice City,play,1.5,0
199998,128470551,RUSH,purchase,1.0,0


In [5]:
def prepare_data():
    steam_raw = pd.read_csv("steam-200k.csv", header=None, names=["User_ID", "Game", "Interaction", "Hours", "Ignore"])
    steam = steam_raw.drop("Ignore", axis =1)
    steam_purchase = steam[steam['Interaction'] == "purchase"]
    steam_play = steam[steam['Interaction'] == "play"]
    steam_pp = pd.merge(steam_purchase, steam_play, how = 'left', left_on = ['User_ID', 'Game'], right_on = ['User_ID', 'Game'])
    steam_pp['Hours_y'] = steam_pp['Hours_y'].fillna(0)
    steam_clean = steam_pp.drop(['Interaction_x', 'Interaction_y', 'Hours_x'], axis = 1)
    steam_clean_rare = steam_clean.groupby('Game').filter(lambda x : len(x) > 10)
    steam_clean_rare['quartile_rating'] = steam_clean_rare.groupby(['Game'])['Hours_y'].rank(pct=True).transform(
        lambda x: pd.qcut(x, 5, labels=range(1,6), duplicates='drop')
    )
    return steam_clean_rare
steam = prepare_data()

In [102]:
# from itertools import combinations, permutations
# from random import choice, sample
# list1 = steam.User_ID.unique()
# list1 = list1.tolist()
# probki = []
# for i in range(10):
#     probki.append(sample(list1, 10))
# probki
# with open('samples_2.txt', 'w') as f:
#     for p in probki:
#         f.write(', '.join(map(str, p)))
#         f.write('\n')
        

In [237]:
from typing import List
class EmbeddingRecommender:
    def __init__(self, data):
        data_listed = data.groupby("User_ID").agg({"Game": lambda x: list(x)})
        sentences = data_listed.Game.to_list()
        sentences

        # users-games
        self.sentences = sentences
        self.data_listed = data_listed
        self.model = None

    def fit(self, window=5, min_count=5, workers=3, sg=0):
        self.model = Word2Vec(self.sentences, window=window, min_count=min_count, workers=workers, sg=sg)

    def recommmend(self, user_tag, n_similar = 10) -> List:
        if self.model is None:
            raise Exception('You need to train first')
        played_games = self.data_listed.loc[user_tag].to_list()
        games = {}
        for game in played_games[0]:
            similar = self.similar_items(game, n_similar)
            for item in similar:
                if item not in played_games[0]:
                    if item[0] not in games:
                        games[item[0]] = (item[1], 1)
                    else:
                        games[item[0]] = (games[item[0]][0] + item[1], games[item[0]][1] + 1)
        for name, (quality, n) in games.items():
            games[name] = quality / n
        best_games = sorted(games.items(), key=lambda x: x[1], reverse=True)[:n_similar]
        xd = [g[0] for g in best_games]
        return xd

    def similar_items(self, item_name, n_similar = 10):
        if self.model is None:
            raise Exception('You need to train first')
        return self.model.wv.most_similar(item_name, topn=n_similar)

In [ ]:
with open('samples.txt', 'r') as f:
    lines = [line.strip() for line in f]

users = []
for line in lines:
    users.append(line.split(', '))
#users

AttributeError: 'list' object has no attribute 'loc'

In [238]:
model = EmbeddingRecommender(steam)
model.fit(window=3, sg=0)


In [247]:
results2 = []
for sample in users:
    przewidywania = {}
    for user in sample:
        przewidywania[int(user)] = model.recommmend(int(user))
    #print(przewidywania)
    results2.append(przewidywania)

In [246]:
print(model.data_listed.loc[8100868].to_list())
for r in results2:
    if '8100868' in r:
        print(r['8100868'])

[['Counter-Strike', 'Day of Defeat', 'Deathmatch Classic', 'Half-Life', 'Half-Life Blue Shift', 'Half-Life Opposing Force', 'Ricochet', 'Team Fortress Classic']]
['Half-Life Deathmatch Source', 'Counter-Strike Condition Zero', 'Half-Life Source', 'Counter-Strike Condition Zero Deleted Scenes', 'Day of Defeat', 'Zombie Panic Source', 'Killing Floor Mod Defence Alliance 2', 'Dead Space', 'Counter-Strike Source', 'Darwinia']


In [240]:
with open('results2.txt', 'w') as f:
    f.write('[')
    f.write(", ".join(map(str, results2)))
    f.write(']')


[('Dota 2', 0.9745949506759644), ("Garry's Mod", 0.972614049911499), ('DC Universe Online', 0.9693325757980347), ('Counter-Strike Global Offensive', 0.964198112487793), ('H1Z1', 0.9606828093528748), ('AdVenture Capitalist', 0.9390743374824524), ('Rust', 0.9366994500160217), ('PAYDAY 2', 0.9311463236808777), ('Clicker Heroes', 0.9282840490341187), ('Call of Duty World at War', 0.9264910817146301)]
[('Dota 2', 0.9612736105918884), ('Ace of Spades', 0.9452863931655884), ('MapleStory', 0.9441447257995605), ('Serious Sam HD The Second Encounter', 0.9394935965538025), ('Infestation Survivor Stories', 0.9384415149688721), ('DC Universe Online', 0.9337234497070312), ('Counter-Strike Global Offensive', 0.9334918260574341), ('Geometry Dash', 0.9320390224456787), ('Echo of Soul', 0.931488573551178), ('Reign Of Kings', 0.9292135834693909)]


# Training and evaluating